### We will colect text data using Twitter API.

In [3]:
# communicate with twitter api
!pip install tweepy

In [4]:
# read or write excell file
!pip install openpyxl     

In [5]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler  # communicate with twitter

In [6]:
consumer_key = "cEQU7HP8sIz5q598uA65g9ooa"
consumer_secret = "GLPgPqHoKyvQTNC5m0Fyd7JaSDE8ONPFKOIjB1dpBuWWuFhVo1"
access_token = "1321518103482695680-7WmynEdqVf8ozt0qYteptjpHejBs30"
access_token_secret = "qHTjoZdsvsAOYDAED7PahuMuNWyrZL0hquUFnSUK4HwMW"

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [10]:
query = "bjp election in bihar"

In [11]:
Tweets = api.search(query, count = 10, lang = 'en', exclude = 'retweets', tweet_mode = 'extended')
# tweet_mode = 'extended' - full text attribute
# tweet_mode = 'compat' - extract tweets having only 140 characters

for tweet in Tweets:
    print(tweet,"\n")
    print("........................................\n")

Status(_api=<tweepy.api.API object at 0x00000294BF7D1A48>, _json={'created_at': 'Thu Oct 29 14:04:59 +0000 2020', 'id': 1321815298236821505, 'id_str': '1321815298236821505', 'full_text': '@SalmanNizami_ Bihar learnt a lesson from the last election when one of the party of Grand Alliance allied with BJP after the election. \nThis may happen too this time, it may be RJD or Congress MLAs as had happened in other States.', 'truncated': False, 'display_text_range': [15, 231], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'SalmanNizami_', 'name': 'Salman Nizami', 'id': 83586157, 'id_str': '83586157', 'indices': [0, 14]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1321792060328865792, 'in_reply_to_status_id_str': '1321792060328865792', 'in_reply_to_user_id': 83586157, 'in_reply_to_user_id_str': '83586157', 'in_reply_

### Get Tweets and some Attributes

tweepy.Cursor() - allow us to get as many tweets as we want

In [12]:
df = pd.DataFrame(columns= ['Tweets', 'Users', 'User_status_count', 'user_followers',
                           'user_location', 'user_verified', 'fav_count', 'rt_count',
                           'tweet_date'])
print(df)
print(df.shape)

Empty DataFrame
Columns: [Tweets, Users, User_status_count, user_followers, user_location, user_verified, fav_count, rt_count, tweet_date]
Index: []
(0, 9)


In [13]:
def stream(data, file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_status_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'user_location'] = tweet.user.location
        df.loc[i, 'user_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [15]:
stream(data=['bjp election in bihar'],file_name='my_tweets')

In [16]:
df.head()

,Tweets,Users,User_status_count,user_followers,user_location,user_verified,fav_count,rt_count,tweet_date,User
0,RT @pallavict: Those angry with BJP for police...,NaN,28090,221,भारत,False,0,48,2020-10-29 14:06:15,पार्थ कुलकर्णी 🇮🇳
1,@SalmanNizami_ Bihar learnt a lesson from the ...,NaN,4402,584,,False,0,0,2020-10-29 14:04:59,Nomanuddin Hassan
2,@ZarrarKhuhro @Khalil_a_hassan Bihar Elections...,NaN,17029,256,Pakistan,False,0,0,2020-10-29 14:00:00,30 Beat Khan
3,@AbhishBanerj Bihar...After Maharashtra it's t...,NaN,4387,673,"Mumbai, India",False,1,0,2020-10-29 13:58:34,Onc Doc
4,RT @pallavict: Those angry with BJP for police...,NaN,8030,68,Hell,False,0,48,2020-10-29 13:58:28,King Of Hell


In [17]:
!pip install textblob 

#### return sentiment(polarity, subjectivity)

#### Polarity score : [-1, +1]

#### Subjectivity : [0.0, 1.0]

#### where 0.0 is very objectivity and 1.0 is very subjective

In [18]:
from textblob import TextBlob

In [19]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ',tweet).split())

In [20]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [21]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [22]:
df.head()

,Tweets,Users,User_status_count,user_followers,user_location,user_verified,fav_count,rt_count,tweet_date,User,clean_tweet
0,RT @pallavict: Those angry with BJP for police...,NaN,28090,221,भारत,False,0,48,2020-10-29 14:06:15,पार्थ कुलकर्णी 🇮🇳,RT Those angry with BJP for police atrocities ...
1,@SalmanNizami_ Bihar learnt a lesson from the ...,NaN,4402,584,,False,0,0,2020-10-29 14:04:59,Nomanuddin Hassan,Bihar learnt a lesson from the last election w...
2,@ZarrarKhuhro @Khalil_a_hassan Bihar Elections...,NaN,17029,256,Pakistan,False,0,0,2020-10-29 14:00:00,30 Beat Khan,a hassan Bihar Elections Modi needed his minio...
3,@AbhishBanerj Bihar...After Maharashtra it's t...,NaN,4387,673,"Mumbai, India",False,1,0,2020-10-29 13:58:34,Onc Doc,Bihar After Maharashtra it s the second electi...
4,RT @pallavict: Those angry with BJP for police...,NaN,8030,68,Hell,False,0,48,2020-10-29 13:58:28,King Of Hell,RT Those angry with BJP for police atrocities ...


In [23]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x))
df.head()

,Tweets,Users,User_status_count,user_followers,user_location,user_verified,fav_count,rt_count,tweet_date,User,clean_tweet,Sentiment
0,RT @pallavict: Those angry with BJP for police...,NaN,28090,221,भारत,False,0,48,2020-10-29 14:06:15,पार्थ कुलकर्णी 🇮🇳,RT Those angry with BJP for police atrocities ...,Negative
1,@SalmanNizami_ Bihar learnt a lesson from the ...,NaN,4402,584,,False,0,0,2020-10-29 14:04:59,Nomanuddin Hassan,Bihar learnt a lesson from the last election w...,Positive
2,@ZarrarKhuhro @Khalil_a_hassan Bihar Elections...,NaN,17029,256,Pakistan,False,0,0,2020-10-29 14:00:00,30 Beat Khan,a hassan Bihar Elections Modi needed his minio...,Neutral
3,@AbhishBanerj Bihar...After Maharashtra it's t...,NaN,4387,673,"Mumbai, India",False,1,0,2020-10-29 13:58:34,Onc Doc,Bihar After Maharashtra it s the second electi...,Negative
4,RT @pallavict: Those angry with BJP for police...,NaN,8030,68,Hell,False,0,48,2020-10-29 13:58:28,King Of Hell,RT Those angry with BJP for police atrocities ...,Negative


In [24]:
n = 1
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 @SalmanNizami_ Bihar learnt a lesson from the last election when one of the party of Grand Alliance allied with BJP… https://t.co/h5Gz3cLb0B

Clean tweet:
 Bihar learnt a lesson from the last election when one of the party of Grand Alliance allied with BJP

Sentiment tweet:
 Positive


In [25]:
n = 25
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 RT @pallavict: Those angry with BJP for police atrocities committed in Munger in Bihar when Election Commission was in charge

Pls remember…

Clean tweet:
 RT Those angry with BJP for police atrocities committed in Munger in Bihar when Election Commission was in charge Pls remember

Sentiment tweet:
 Negative


In [26]:
n = 500
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 #ElectionswithMC 🗳️ "People of Bihar have taken a resolve to again defeat those who brought 'jungle raj' in the sta… https://t.co/hrrl85VdqK

Clean tweet:
 ElectionswithMC People of Bihar have taken a resolve to again defeat those who brought jungle raj in the sta

Sentiment tweet:
 Neutral


In [27]:
n = 777
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 @ArnabGoswamiRTV @narendramodi @AmitShah @myogiadityanath due to implementation of code of conduct(aachaar sanhita)… https://t.co/WXI1ZSH4la

Clean tweet:
 due to implementation of code of conduct aachaar sanhita

Sentiment tweet:
 Negative


In [28]:
df[df.Sentiment == 'Positive'].shape[0]

349

In [29]:
df[df.Sentiment == 'Neutral'].shape[0]

480

In [30]:
df[df.Sentiment == 'Negative'].shape[0]

171